<a href="https://colab.research.google.com/github/JSJeong-me/KOSA_ChatGPT_0531/blob/main/Langchain_EX_0530.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LangChain이란?
* Language : Large Language Model의 Language
* Chain: 모듈들을 체인으로 연결하여 다른 툴의 출력을 다음 툴의 입력으로 연결하는 방식
  * 예: 검색 -> OpenAI(LLM) -> 계산
* LLM(Large Language Model)을 이용해서 뭔가 만들때 쓸수 있는 툴/라이브러리 모음.
* 파이썬과 자바스크립트 라이브러리를 제공.
* 검색, 메모리, 계산, 요약, 색인, Agent(에이전트) 등의 기능을 제공
* 모듈화로 쉽게 부품을 갈아끼울수 있음
* LLM은 기본 OpenAI (text-davinci-003) 이용

---
참고 자료:
* 파이썬 문서: https://python.langchain.com/en/latest/index.html
* 개념 문서: https://docs.langchain.com/docs/
* 소스: https://github.com/hwchase17/langchain

---

In [1]:
#@title 기본 패키지(openai, langchain) 설치
!pip install openai
!pip install langchain

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 47.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 19.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 939.3/939.3 kB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 4.7 MB/s eta 0:00:00


In [2]:
#@title 기타 패키지 설치 (구글검색, 위키피디아, VectorStore, HuggingFace Embedding)
!pip install google-search-results
!pip install wikipedia
!pip install faiss-cpu # 오픈소스 벡터DB (Facebook, MIT license)
!pip install sentence_transformers # HuggingFace Embedding 사용 위해서 필요
!pip install tiktoken # Summarization 할때 필요

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32002 sha256=91886fea13139255be255328d3c279642efe59bf7588a081419d5acb6c5315e0
  Stored in directory: /root/.cache/pip/wheels/d3/b2/c3/03302d12bb44a2cdff3c9371f31b72c0c4e84b8d2285eeac53
Successfully built google-search-results
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11680 sha256=4c3c596973da8f03550a62592b4fb5217e692d5b2bd058dc870ac38d691f887e
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
 

In [ ]:
#@title 0. API 키 설정
import os
#@markdown https://platform.openai.com/account/api-keys
OPENAI_API_KEY = "sk-" #@param {type:"string"}

#@markdown https://huggingface.co/settings/tokens
#@markdown HuggingFace에서 모델 다운로드나 클라우드 모델 사용하기 위해서 필요 (무료)
HUGGINGFACEHUB_API_TOKEN = "" #@param {type:"string"}

#@markdown https://serpapi.com/manage-api-key
#@markdown 구글 검색하기 위해서 필요 (월 100회 무료)
SERPAPI_API_KEY = "" #@param {type:"string"}

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN
os.environ["SERPAPI_API_KEY"] = SERPAPI_API_KEY


In [7]:
#@title 1. OpenAI LLM (text-davinci-003)

from langchain.llms import OpenAI

llm = OpenAI(model_name='text-davinci-003', temperature=0.9)

llm('미국의 빌보드차트에 오른 한국 가수 이름을 알려 주세요.')

'\n\n- PSY (싸이) - Gangnam Style, Gentleman\n- BTS (방탄소년단) - Boy With Luv, DNA\n- BLACKPINK (블랙핑크) - Kill This Love, DDU-DU DDU-DU\n- Taeyeon (태연) - I, Fine\n- IU (아이유) - Palette, BBIBBI'

In [9]:
#@title 2. ChatOpenAI LLM (gpt-3.5-turbo)

from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

chat = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.9)
sys = SystemMessage(content="당신은 음악 추천을 해주는 전문 AI입니다.")
msg = HumanMessage(content='미국의 빌보드차트에 오른 한국 가수 노래를 찾아 주세요.')

aimsg = chat([sys, msg])
aimsg.content

'현재(2021년 10월 11일 기준) 빌보드 차트에는 아래와 같은 한국 가수의 노래들이 오르고 있습니다.\n\n1. 방탄소년단 (BTS) - Butter (#11)\n2. 방탄소년단 (BTS) - Permission to Dance (#33)\n3. 방탄소년단 (BTS) - Dynamite (#45)\n4. 송민호 (MINO) - Run Away (#79)\n\n위의 노래들 중 원하는 노래가 있으시면 말씀해주세요!'

In [10]:
#@title 3. Prompt Template & chain

from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    input_variables=["상품"],
    template="{상품} 만드는 회사 이름 추천해줘. 기억에 남는 한글 이름으로",
)

prompt.format(상품="AI 여행 추천 서비스")

'AI 여행 추천 서비스 만드는 회사 이름 추천해줘. 기억에 남는 한글 이름으로'

In [11]:
from langchain.chains import LLMChain
chain = LLMChain(llm=chat, prompt=prompt)

# chain.run("AI 여행 추천 서비스")
chain.run(상품="AI 여행 추천 서비스")

'하늘길여행 (Haneulgilyeohaeng)'

In [12]:
#@title 4. ChatPromptTemplate & chain

from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

chat = ChatOpenAI(temperature=0)

template="You are a helpful assisstant that tranlates {input_language} to {output_language}."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template="{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])


chatchain = LLMChain(llm=chat, prompt=chat_prompt)
chatchain.run(input_language="English", output_language="Korean", text="I love programming.")


'저는 프로그래밍을 좋아합니다.'

In [13]:
#@title 5. Agents and Tools

from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType

* Tool: 특정 일을 하는 역할. 예) 구글 검색, 디비 조회, Python 실행/계산.
* Agents: 어떤 툴을 어떤 순서로 실행할지 결정하는 역할 (LLM 이용)

In [14]:
# tools = load_tools(["serpapi", "llm-math"], llm=chat)
tools = load_tools(["wikipedia", "llm-math"], llm=chat)

agent = initialize_agent(tools, llm=chat, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [22]:
agent.run("How many members are in Korea's 'BTS' group? What is the sum of the ages of all members as of today, May 30, 2023?") # 한국의 'New Jeans' 그룹 멤버의 인원 수는? 오늘 2023년 5월 30일 현재 전체 멤버들의 나이의 합은 얼마인가요



> Entering new AgentExecutor chain...
I need to find information about the BTS group and their members' ages.
Action: Wikipedia
Action Input: "BTS group"
Observation: Page: BTS
Summary: BTS (Korean: 방탄소년단; RR: Bangtan Sonyeondan; lit. Bulletproof Boy Scouts), also known as the Bangtan Boys, is a South Korean boy band formed in 2010. The band consists of Jin, Suga, J-Hope, RM, Jimin, V, and Jungkook, who co-write or co-produce much of their material. Originally a hip hop group, their musical style has evolved to incorporate a wide range of genres, while their lyrics have focused on subjects including mental health, the troubles of school-age youth and coming of age, loss, the journey towards self-love, individualism, and the consequences of fame and recognition. Their discography and adjacent work has also referenced literature, philosophy and psychological concepts, and includes an alternate universe storyline.
BTS debuted in 2013 under Big Hit Entertainment with the single album 2 C

'There are 7 members in BTS and the sum of their ages as of May 30, 2023 is 190.'

In [23]:
agent.tools

[WikipediaQueryRun(name='Wikipedia', description='A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.', args_schema=None, return_direct=False, verbose=False, callbacks=None, callback_manager=None, api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/usr/local/lib/python3.10/dist-packages/wikipedia/__init__.py'>, top_k_results=3, lang='en', load_all_available_meta=False, doc_content_chars_max=4000)),
 Tool(name='Calculator', description='Useful for when you need to answer questions about math.', args_schema=None, return_direct=False, verbose=False, callbacks=None, callback_manager=None, func=<bound method Chain.run of LLMMathChain(memory=None, callbacks=None, callback_manager=None, verbose=False, llm_chain=LLMChain(memory=None, callbacks=None, callback_manager=None, verbose=False, prompt=PromptTemplate(input_variables=['question'], 

In [24]:
print(agent.tools[0].description)
print(agent.tools[1].description)

A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.
Useful for when you need to answer questions about math.


In [25]:
#@title 6. Memory
from langchain import ConversationChain

conversation = ConversationChain(llm=chat, verbose=True)
conversation.predict(input="인공지능에서 Transformer가 뭐야?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: 인공지능에서 Transformer가 뭐야?
AI:

> Finished chain.


'Transformer는 자연어 처리 분야에서 매우 유용한 딥러닝 모델 중 하나입니다. 이 모델은 기계 번역, 질의응답, 요약 등 다양한 자연어 처리 작업에서 사용됩니다. Transformer는 기존의 RNN, LSTM 등의 모델보다 더욱 빠르고 정확한 결과를 제공합니다. 이 모델은 구글에서 개발되었으며, 현재까지도 많은 연구자들이 이를 활용하여 다양한 자연어 처리 연구를 수행하고 있습니다.'

In [26]:
conversation.predict(input="RNN하고 차이 설명해줘.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 인공지능에서 Transformer가 뭐야?
AI: Transformer는 자연어 처리 분야에서 매우 유용한 딥러닝 모델 중 하나입니다. 이 모델은 기계 번역, 질의응답, 요약 등 다양한 자연어 처리 작업에서 사용됩니다. Transformer는 기존의 RNN, LSTM 등의 모델보다 더욱 빠르고 정확한 결과를 제공합니다. 이 모델은 구글에서 개발되었으며, 현재까지도 많은 연구자들이 이를 활용하여 다양한 자연어 처리 연구를 수행하고 있습니다.
Human: RNN하고 차이 설명해줘.
AI:

> Finished chain.


'RNN은 Recurrent Neural Network의 약자로, 시퀀스 데이터를 처리하는 데 특화된 딥러닝 모델입니다. RNN은 이전 시점의 출력값을 현재 시점의 입력값으로 사용하여 시퀀스 데이터를 처리합니다. 하지만 RNN은 시퀀스가 길어질수록 기울기 소실 문제가 발생하여 성능이 저하됩니다. 반면 Transformer는 Self-Attention 메커니즘을 사용하여 입력 시퀀스의 모든 위치를 동시에 고려할 수 있으며, 이를 통해 기울기 소실 문제를 해결합니다. 또한 Transformer는 병렬 처리가 가능하여 RNN보다 더욱 빠른 속도로 처리할 수 있습니다.'

In [27]:
conversation.memory

ConversationBufferMemory(chat_memory=ChatMessageHistory(messages=[HumanMessage(content='인공지능에서 Transformer가 뭐야?', additional_kwargs={}, example=False), AIMessage(content='Transformer는 자연어 처리 분야에서 매우 유용한 딥러닝 모델 중 하나입니다. 이 모델은 기계 번역, 질의응답, 요약 등 다양한 자연어 처리 작업에서 사용됩니다. Transformer는 기존의 RNN, LSTM 등의 모델보다 더욱 빠르고 정확한 결과를 제공합니다. 이 모델은 구글에서 개발되었으며, 현재까지도 많은 연구자들이 이를 활용하여 다양한 자연어 처리 연구를 수행하고 있습니다.', additional_kwargs={}, example=False), HumanMessage(content='RNN하고 차이 설명해줘.', additional_kwargs={}, example=False), AIMessage(content='RNN은 Recurrent Neural Network의 약자로, 시퀀스 데이터를 처리하는 데 특화된 딥러닝 모델입니다. RNN은 이전 시점의 출력값을 현재 시점의 입력값으로 사용하여 시퀀스 데이터를 처리합니다. 하지만 RNN은 시퀀스가 길어질수록 기울기 소실 문제가 발생하여 성능이 저하됩니다. 반면 Transformer는 Self-Attention 메커니즘을 사용하여 입력 시퀀스의 모든 위치를 동시에 고려할 수 있으며, 이를 통해 기울기 소실 문제를 해결합니다. 또한 Transformer는 병렬 처리가 가능하여 RNN보다 더욱 빠른 속도로 처리할 수 있습니다.', additional_kwargs={}, example=False)]), output_key=None, input_key=None, return_messages=False, human_prefix='Human', ai_prefix='AI', memory_key='history'

* ConversationBufferMemory : 대화 기록(기본)
* ConversationBufferWindowMemory : 마지막 n개의 대화만 기억
* Entity Memory : 개체에 대한 정보를 저장
* Conversation Knowledge Graph Memory: 개체의 triple 저장: (sam, 좋아하는 색, 파랑)
* ConversationSummaryMemory : 대화의 요약본을 저장
* ConversationSummaryBufferMemory : 대화 요약본 + 마지막 n토큰 기억
* ConversationTokenBufferMemory : 마지막 n토큰 기억
* VectorStore-Backed Memory : 벡터DB에 정보 저장

In [28]:
#@title 7. Document Loaders
from langchain.document_loaders import WebBaseLoader

loader = WebBaseLoader(web_path="https://ko.wikipedia.org/wiki/NewJeans")

documents = loader.load()

In [29]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)
len(docs)

# 4096 token = 3000 English word

13

In [30]:
print(docs[1].page_content)

한국어














읽기

편집

역사 보기









도구





도구
사이드바로 이동
숨기기



		동작
	

읽기편집역사 보기




		일반
	

여기를 가리키는 문서가리키는 글의 최근 바뀜파일 올리기특수 문서 목록고유 링크문서 정보이 문서 인용하기위키데이터 항목




		인쇄/내보내기
	

책 만들기PDF로 다운로드인쇄용 판




		다른 프로젝트
	

위키미디어 공용




















위키백과, 우리 모두의 백과사전.




  EP에 대해서는 New Jeans (EP) 문서를 참고하십시오.


NewJeans뉴진스








기본 정보


결성 지역

대한민국 서울특별시


장르

K-pop댄스 팝


활동 시기

2022년 7월 22일 ~


레이블

하이브


소속사

어도어


웹사이트

공식 홈페이지


구성원


민지하니다니엘해린혜인


NewJeans(뉴진스)는 2022년 7월 22일에 데뷔한 대한민국의 5인조 걸 그룹으로, 소속사는 하이브 산하의 레이블인 어도어이다. SM 엔터테인먼트 디렉터 출신으로 하이브에 영입된 민희진이 프로듀서로 나서서 발굴한 걸그룹이다. 2022년 8월 18일 《엠카운트다운》에서 데뷔 3주만에 첫 1위를 차지했다.


활동[편집]
이 문단은 영어 위키백과의 NewJeans 문서를 번역하여 문서의 내용을 확장할 필요가 있습니다.중요한 번역 안내를 보려면 [펼치기]를 클릭하십시오.
신뢰성 있고 확인할 수 있는 출처가 제시되도록 번역하여 주십시오.
번역을 완료한 후에는 {{번역된 문서}} 틀을 토론창에 표기하여 저작자를 표시하여 주십시오.
문맥상 이해를 돕기 위해 관련 문서를 같이 번역해주시는 것이 좋습니다.
번역을 확장할 필요가 있는 내용이 포함된 다른 문서를 보고 싶으시다면 분류:번역 확장 필요 문서를 참고해주세요.
데뷔 전[편집]
민지와 하니는 방탄소년단의 Permission to Dance 뮤직비디오에 출연하였다.


---
* Web Page
* PDF
* Email
* Twitter
* DataFrame (pandas)
* YouTube
* Notion
* Google Drive
* Powerpoint, Word, 
* ...
---

In [31]:
#@title 8. Summarization
from langchain.chains.summarize import load_summarize_chain
chain = load_summarize_chain(chat, chain_type="map_reduce", verbose=True)
chain.run(docs[:3])




> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"NewJeans - 위키백과, 우리 모두의 백과사전






































본문으로 이동








주 메뉴





주 메뉴
사이드바로 이동
숨기기



		둘러보기
	

대문최근 바뀜요즘 화제임의의 문서로기부




		사용자 모임
	

사랑방사용자 모임관리 요청




		편집 안내
	

도움말정책과 지침질문방




언어

이 위키백과에서 언어 링크는 문서 제목 건너편의 문서 최상단에 있습니다. 최상단으로 이동합니다.



















검색















계정 만들기로그인






개인 도구




 계정 만들기 로그인




		로그아웃한 편집자를 위한 문서 더 알아보기


기여토론


























목차
사이드바로 이동
숨기기




처음 위치





1활동


				활동 하위섹션 토글하기
			




1.1데뷔 전







1.22022년









2구성원







3역사


				역사 하위섹션 토글하기
			




3.12019–2021: 결성 및 데뷔 전 활동









4음반


				음반 하위섹션 토글하기
			




4.1EP







4.2싱글









5음반 외 활동







6수상 목록


				수상 목록 하위섹션 토글하기
			




6.1가요 프로그램 1위









7기타 시상식







8각주







9외부 링크













목차 토글






목차 토글







NewJeans




21개 언어



العربيةБеларускаяDeutschEnglishEspaño

'NewJeans is a South Korean girl group that debuted in July 2022 under the label of HIVE\'s subsidiary, Adore. The group consists of five members and was produced by Min Hee-jin. They have won awards on various music programs and two members appeared in BTS\'s "Permission to Dance" music video before their debut.'

* "stuff": LLM 한번에 다 보냄. 길면 오류남
* "map_reduce": 나눠서 요약, 전체 요약본 다시 요약
* "refine": (요약 + 다음 문서) => 요약
* "map_rerank": 점수매겨서 중요한거로 요약

In [32]:
#@title 9. Embeddings and VectorStore
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.embeddings import OpenAIEmbeddings

# embeddings = OpenAIEmbeddings()
embeddings = HuggingFaceEmbeddings()

from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import FAISS

# from langchain.text_splitter import RecursiveCharacterTextSplitter
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

index = VectorstoreIndexCreator(
    vectorstore_cls=FAISS,
    embedding=embeddings,
    # text_splitter=text_splitter,
    ).from_loaders([loader])

# 파일로 저장
index.vectorstore.save_local("faiss-nj")

* Embedding : LLM에 input(text)를 넣었을때 나오는 특징 벡터 - Neural Network의 특정 메모리를 읽어서 만들고, 보통 512개 이상의 float로 만들어짐.
* 의미상 비슷한 input이 들어갔을때, 나오는 특징 벡터가 유사한 특징을 갖고, 거리상 가깝다.
* OpenAI에서 text를 넣으면 Embedding을 돌려주는 API를 제공한다. (다만 비용 발생)
* 로컬에서 유사한 알고리즘을 돌릴수 있다. (다만 똑같은 벡터는 아니고, openAI 임베딩과는 비교할수 없다)
* HuggingFaceEmbeddings 사용하면 로컬에서 돌릴수 있고, GPU 있다면 가속도 지원함.
* Vector Database는 Embedding 벡터와 텍스트를 저장하는 DB
  * Pinecone 서비스 (유료, 무료 제한적)
  * FAISS (페이스북, 오픈소스, 로컬)
  * Embeddings -> 텍스트를 저장
  * Embeddings로 검색할수 있으며, 거리상 가장 가까운 항목을 가져올수 있음. (거리를 구하는 방법은 여러가지)
  * 유사한 의미를 갖는 텍스트에 대한 Embedding 벡터는 거리상 가깝다.

In [33]:
index.query("뉴진스의 데뷔곡은?", llm=chat, verbose=True)



> Entering new RetrievalQA chain...

> Finished chain.


'뉴진스의 데뷔곡은 "Attention"입니다.'

In [34]:
index.query("뉴진스의 데뷔 멤버는?", llm=chat, verbose=True)



> Entering new RetrievalQA chain...

> Finished chain.


'뉴진스(NewJeans)는 2022년 7월 22일에 데뷔한 대한민국의 5인조 걸 그룹입니다. 멤버는 민지, 하니, 다니엘, 해린, 혜인으로 구성되어 있습니다.'

In [35]:
index.query("멤버의 나이는?", llm=chat, verbose=True)



> Entering new RetrievalQA chain...

> Finished chain.


'위키백과에서는 멤버들의 나이 정보가 제공되지 않고 있습니다. 따라서, 현재로서는 멤버들의 나이에 대한 정보를 알 수 없습니다.'

In [36]:
index.query("멤버의 나이는? (오늘은 2023년 5월 30일)", llm=chat, verbose=True)



> Entering new RetrievalQA chain...

> Finished chain.


'민지는 2004년 5월 7일생으로 19세, 하니는 2004년 10월 6일생으로 18세, 다니엘은 2005년 4월 11일생으로 18세, 해린은 2006년 5월 15일생으로 17세, 혜인은 2008년 4월 21일생으로 15세입니다. (오늘은 2023년 5월 30일 기준)'

In [37]:
#@title FAISS 벡터DB 디스크에서 불러오기
from langchain.indexes.vectorstore import VectorStoreIndexWrapper

fdb = FAISS.load_local("faiss-nj", embeddings)
index2 = VectorStoreIndexWrapper(vectorstore=fdb)

In [38]:
index2.query("뉴진스의 데뷔 멤버는?", llm=chat, verbose=True)



> Entering new RetrievalQA chain...

> Finished chain.


'뉴진스(NewJeans)는 2022년 7월 22일에 데뷔한 대한민국의 5인조 걸 그룹입니다. 멤버는 민지, 하니, 다니엘, 해린, 혜인으로 구성되어 있습니다.'